In [1]:
import pandas as pd
import numpy as np
import os
import re
from icecream import ic
from collections import defaultdict
import warnings
pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")
# import pickle

/tmp/ipykernel_3967800/2563711703.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [63]:
# embeds_file = '/home/varunaja/mri_pet/ready_data/ADNI_ABTAU_MOREFEATURES.pkl'
# with open(embeds_file, 'rb') as file:
#     # Load the data from the pickle file
#     data = pickle.load(file)

In [2]:
def code_data_type(df, meta, percent_missing):
    for i, col in enumerate(df.columns):
        if col == 'ID' or col == 'LONIUID.T1' or col == 'LONIUID.FLAIR':
            continue
        # print(col)
        # print(meta1[meta1['name'] == col]['type'].iloc[0])
        meta[col] = {}
        meta[col]['name'] = col
        meta[col]['unique_values'] = list(df[col].dropna().unique())
        meta[col]['num_unique_values'] = len(meta[col]['unique_values'])
        meta[col]['length'] = 1
        mydtype = df[col].dtype
        if 'category' in str(mydtype):
            meta[col]['type'] = 'C'
        else:
            meta[col]['type'] = 'N'
        if col == 'img_MRI_T1':
            meta[col]['type'] = 'M'
        meta[col]['missing_rate'] = round(percent_missing[col], 2)
    df = pd.DataFrame(meta).T[['name', 'missing_rate', 'type', 'num_unique_values', 'length', 'unique_values']]
    return df

# Load Data

In [3]:
# data = pd.read_csv('/home/varunaja/mri_pet/adrd_tool_varuna/adrd_transformer/data/adni_data_single.csv')
data = pd.read_csv('/home/varunaja/mri_pet/adrd_tool_varuna/adrd_transformer/data/oasis_data_single.csv')

In [4]:
data.drop(columns = ['cd_DIAGNOSIS'], inplace = True)

In [5]:
demog = data.columns[data.columns.str.contains('his')].to_list()
# diagnosis = ['cd_DIAGNOSIS']
mmse = data.columns[data.columns.str.contains('bat') & data.columns.str.contains('MMSE')].to_list()
cdr = data.columns[data.columns.str.contains('cdr')].to_list()
cvd = data.columns[data.columns.str.contains('cvd')].to_list()
physical = data.columns[data.columns.str.contains('ph')].to_list()
moca = data.columns[data.columns.str.contains('bat') & data.columns.str.contains('MOCA')].to_list()
np = data.columns[(data.columns.str.contains('bat')) & (~data.columns.str.contains('MOCA')) & (~data.columns.str.contains('MMSE'))].to_list()
gds = data.columns[data.columns.str.contains('gds')].to_list()
npiq = data.columns[data.columns.str.contains('npiq')].to_list()
faq = data.columns[data.columns.str.contains('faq')].to_list()
hx = data.columns[data.columns.str.contains('iha')].to_list()
labels = ['amy_label', 'tau_label', 'NC', 'MCI', 'DE']
volumes = data.columns[data.columns.str.contains('FS')].to_list()

# ADNI

In [8]:
all_columns = demog + mmse + volumes + cdr + cvd + physical + moca + np + gds + npiq + faq + hx + labels

numerical = ['his_EDUC'] + mmse + physical + volumes + cdr + np + ['his_NACCAGE', 'bat_MOCANAMI', 'bat_MOCASER7', 'bat_MOCARECN', 'bat_NACCMOCA', 'bat_TRAILA', 'bat_TRAILB', 'gds_NACCGDS', 'npiq_NPISCORE', 'faq_FAQTOTAL', 'cvd_HACHIN'] 
# imaging = ['img_MRI_T1']

for col in all_columns:
    if col not in numerical: # and col not in imaging:
        data[col] = data[col].astype('category')

for col in numerical:
    data[col] = pd.to_numeric(data[col], errors='coerce')

In [9]:
print(all_columns)

['his_SEX', 'his_HISPANIC', 'his_PRIMLANG', 'his_EDUC', 'his_MARISTAT', 'his_RESIDENC', 'his_HANDED', 'his_NACCNIHR', 'his_NACCAGE', 'his_NACCMOM', 'his_NACCDAD', 'bat_MMSEORDA', 'bat_MMSEORLO', 'bat_NACCMMSE', 'FS_MTL_VOLUME', 'FS_NEO-T_VOLUME', 'FS_TEMPO_PARIETAL_VOLUME', 'FS_FRONTAL_VOLUME', 'cdr_MEMORY', 'cdr_ORIENT', 'cdr_JUDGEMENT', 'cdr_COMMUN', 'cdr_HOMEHOBB', 'cdr_PERSCARE', 'cdr_CDRSUM', 'cdr_CDRGLOB', 'cvd_ABRUPT', 'cvd_STEPWISE', 'cvd_SOMATIC', 'cvd_EMOT', 'cvd_HXHYPER', 'cvd_HXSTROKE', 'cvd_FOCLSYM', 'cvd_FOCLSIGN', 'cvd_HACHIN', 'ph_HEIGHT', 'ph_WEIGHT', 'ph_BPSYS', 'ph_BPDIAS', 'ph_HRATE', 'bat_NACCMOCA', 'bat_MOCATRAI', 'bat_MOCACUBE', 'bat_MOCACLOC', 'bat_MOCACLON', 'bat_MOCACLOH', 'bat_MOCANAMI', 'bat_MOCADIGI', 'bat_MOCALETT', 'bat_MOCASER7', 'bat_MOCAREPE', 'bat_MOCAFLUE', 'bat_MOCAABST', 'bat_MOCARECN', 'bat_MOCAORDT', 'bat_MOCAORMO', 'bat_MOCAORYR', 'bat_MOCAORDY', 'bat_MOCAORPL', 'bat_MOCAORCT', 'bat_BOSTON', 'bat_MEMUNITS', 'bat_ANIMALS', 'bat_TRAILA', 'bat_TRAI

In [10]:
meta = defaultdict()
percent_missing = dict(data.isnull().sum() / len(data))
percent_missing

{'amy_label': 0.0,
 'tau_label': 0.4838300570703868,
 'ID': 0.0,
 'his_SEX': 0.0,
 'his_HISPANIC': 0.003804692454026633,
 'his_PRIMLANG': 0.0,
 'his_EDUC': 0.0,
 'his_MARISTAT': 0.0031705770450221942,
 'his_RESIDENC': 0.01585288522511097,
 'his_HANDED': 0.0006341154090044388,
 'his_NACCNIHR': 0.0057070386810399495,
 'his_NACCAGE': 0.0,
 'his_NACCMOM': 0.6119213696892835,
 'his_NACCDAD': 0.6169942929613189,
 'ph_HEIGHT': 0.8795180722891566,
 'ph_WEIGHT': 0.0063411540900443885,
 'ph_BPSYS': 0.007609384908053266,
 'ph_BPDIAS': 0.007609384908053266,
 'ph_HRATE': 0.010145846544071021,
 'cvd_ABRUPT': 0.18706404565630944,
 'cvd_STEPWISE': 0.18706404565630944,
 'cvd_SOMATIC': 0.186429930247305,
 'cvd_EMOT': 0.18706404565630944,
 'cvd_HXHYPER': 0.186429930247305,
 'cvd_HXSTROKE': 0.186429930247305,
 'cvd_FOCLSYM': 0.186429930247305,
 'cvd_FOCLSIGN': 0.186429930247305,
 'cvd_HACHIN': 0.186429930247305,
 'npiq_DEL': 0.4984147114774889,
 'npiq_HALL': 0.4984147114774889,
 'npiq_AGIT': 0.49904882688

In [11]:
meta = defaultdict()
percent_missing = dict(data.isnull().sum() / len(data))
filtered_percent_missing = {key: value for key, value in percent_missing.items() if value > 0.89}
filtered_percent_missing


{}

In [ ]:
for i, col in enumerate(data.columns):
    mydtype = data[col].dtype
    if 'category' in str(mydtype):
        # print(col)
        print(data[col].value_counts())

In [10]:
df = code_data_type(data, meta, percent_missing)
df.to_csv('../meta_files/adni_meta_file.csv', index=False)

# OASIS

In [6]:
# don't need this, should have removed sooner
data.drop(columns=['bat_CRAFTDTI', 'cvd_STROKCOG', 'cvd_CVDIMAG'], inplace = True)
np.remove('bat_CRAFTDTI')
cvd.remove('cvd_STROKCOG')
cvd.remove('cvd_CVDIMAG')

In [7]:
all_columns = demog + mmse + volumes + cdr + np + cvd + physical + moca + gds + npiq + faq + hx + labels

numerical = ['his_EDUC'] + mmse + physical + volumes + cdr + np + ['his_NACCAGE', 'his_SMOKYRS', 'his_QUITSMOK', 'his_PACKSPER', 'bat_MOCANAMI', 'bat_MOCASER7', 'bat_MOCARECN', 'bat_NACCMOCA', 'gds_NACCGDS', 'faq_FAQTOTAL', 'cvd_HACHIN'] 
# imaging = ['img_MRI_T1']

for col in all_columns:
    if col not in numerical: # and col not in imaging:
        data[col] = data[col].astype('category')

for col in numerical:
    data[col] = pd.to_numeric(data[col], errors='coerce')

In [8]:
meta = defaultdict()
percent_missing = dict(data.isnull().sum() / len(data))
filtered_percent_missing = {key: value for key, value in percent_missing.items() if value > 0.89}

filtered_percent_missing

{}

In [11]:
data['DE'].value_counts() 

DE
0.0    797
1.0    135
Name: count, dtype: int64

In [12]:
df = code_data_type(data, meta, percent_missing)
df

,name,missing_rate,type,num_unique_values,length,unique_values
tau_label,tau_label,0.61,C,2,1,"[0.0, 1.0]"
amy_label,amy_label,0.0,C,2,1,"[0.0, 1.0]"
his_SEX,his_SEX,0.0,C,2,1,"[female, male]"
his_HISPANIC,his_HISPANIC,0.07,C,2,1,"[no, yes]"
his_EDUC,his_EDUC,0.0,N,18,1,"[18.0, 17.0, 16.0, 20.0, 14.0, 12.0, 13.0, 19...."
...,...,...,...,...,...,...
gds_NACCGDS,gds_NACCGDS,0.0,N,13,1,"[0.0, 1.0, 2.0, 3.0, 7.0, 5.0, 10.0, 8.0, 4.0,..."
faq_FAQTOTAL,faq_FAQTOTAL,0.0,N,29,1,"[0.0, 7.0, 3.0, 1.0, 24.0, 10.0, 2.0, 23.0, 13..."
NC,NC,0.02,C,2,1,"[1.0, 0.0]"
MCI,MCI,0.02,C,2,1,"[0.0, 1.0]"


In [13]:
print(all_columns)

['his_SEX', 'his_HISPANIC', 'his_EDUC', 'his_MARISTAT', 'his_LIVSIT', 'his_INDEPEND', 'his_RESIDENC', 'his_HANDED', 'his_NACCAGE', 'his_NACCNIHR', 'his_NACCMOM', 'his_NACCDAD', 'his_TOBAC30', 'his_TOBAC100', 'his_SMOKYRS', 'his_PACKSPER', 'his_QUITSMOK', 'his_CVHATT', 'his_CVAFIB', 'his_CVANGIO', 'his_CVBYPASS', 'his_CVPACE', 'his_CVCHF', 'his_CVOTHR', 'his_CBSTROKE', 'his_CBTIA', 'his_PD', 'his_PDOTHR', 'his_SEIZURES', 'his_NCOTHR', 'his_DIABETES', 'his_HYPERTEN', 'his_HYPERCHO', 'his_B12DEF', 'his_THYROID', 'his_INCONTU', 'his_INCONTF', 'his_ALCOHOL', 'his_ABUSOTHR', 'his_PSYCDIS', 'bat_NACCMMSE', 'FS_MTL_VOLUME', 'FS_NEO-T_VOLUME', 'FS_TEMPO_PARIETAL_VOLUME', 'FS_FRONTAL_VOLUME', 'cdr_MEMORY', 'cdr_ORIENT', 'cdr_JUDGMENT', 'cdr_COMMUN', 'cdr_HOMEHOBB', 'cdr_PERSCARE', 'cdr_CDRSUM', 'cdr_CDRGLOB', 'bat_LOGIMEM', 'bat_MEMUNITS', 'bat_UDSBENTC', 'bat_UDSBENTD', 'bat_UDSBENRS', 'bat_ANIMALS', 'bat_VEG', 'bat_TRAILA', 'bat_TRAILB', 'bat_WAIS', 'bat_UDSVERFC', 'bat_UDSVERFN', 'bat_UDSVERN

In [14]:
for i, col in enumerate(data.columns):
    mydtype = data[col].dtype
    if 'category' in str(mydtype):
        # print(col)
        print(data[col].value_counts())

tau_label
0.0    317
1.0     56
Name: count, dtype: int64
amy_label
0.0    609
1.0    345
Name: count, dtype: int64
his_SEX
female    533
male      421
Name: count, dtype: int64
his_HISPANIC
no     889
yes      1
Name: count, dtype: int64
his_MARISTAT
0.0    645
2.0    123
1.0    114
4.0     55
3.0     10
5.0      6
Name: count, dtype: int64
his_LIVSIT
1.0    635
0.0    231
2.0     59
4.0     25
3.0      4
Name: count, dtype: int64
his_INDEPEND
0.0    891
1.0     53
2.0      8
3.0      1
Name: count, dtype: int64
his_RESIDENC
0.0    907
1.0     43
2.0      1
Name: count, dtype: int64
his_HANDED
1.0    859
0.0     83
2.0      2
Name: count, dtype: int64
his_NACCNIHR
whi    813
blk    131
asi      6
mul      3
ind      1
Name: count, dtype: int64
his_NACCMOM
0.0    448
1.0    409
Name: count, dtype: int64
his_NACCDAD
0.0    615
1.0    212
Name: count, dtype: int64
his_TOBAC30
0.0    527
1.0     37
Name: count, dtype: int64
his_TOBAC100
0.0    307
1.0    251
Name: count, dtype: int64
his_

In [61]:
df.to_csv('../meta_files/oasis_meta_file.csv', index=False)